# Tagging Job description

In [1]:
import pandas as pd       
train = pd.read_csv("train.tsv", header=0, \
                    delimiter="\t", quoting=3)

test = pd.read_csv("test.tsv", header=0, \
                    delimiter="\t", quoting=3)

In [16]:
example1 = train["description"][0]

In [17]:
from nltk.corpus import stopwords # Import the stop word list

from bs4 import BeautifulSoup

def job_to_words( raw_job, lang="english"):
    # Function to convert a raw job posting to a string of words
    # The input is a single string (a raw job description), and 
    # the output is a single string (a preprocessed job description)
    #
    # 1. Remove HTML
    job_text = BeautifulSoup(raw_job, "lxml").get_text() 
    #
    # 1. Remove non-letters        
    letters_only = re.sub(u"[^a-zA-ZàáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸÝ]", " ", job_text) 
    #
    # 2. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 3. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words(lang))                  
    # 
    # 4. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 5. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))


In [6]:

# Get the number of jobs based on the dataframe column size
num_jobs = train["description"].size
# Initialize an empty list to hold the clean jobs
clean_train_jobs = []

print("Cleaning and parsing the training set job descriptions...\n")

# Loop over each job; create an index i that goes from 0 to the length
# of the job list 
for i in range( 0, num_jobs ):
    # Call our function for each one, and add the result to the list of

    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print("Job %d of %d\n" % ( i+1, num_jobs ))                                                                    

    # clean jobs
    clean_train_jobs.append( job_to_words( train["description"][i] ))

Cleaning and parsing the training set job descriptions...

Job 1000 of 4375

Job 2000 of 4375

Job 3000 of 4375

Job 4000 of 4375



In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer1 = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 2000,
                              ngram_range = (1,4)) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer1.fit_transform(clean_train_jobs)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()
# Get the number of jobs based on the dataframe column size
num_data = train["tags"].size

# Initialize an empty list to hold the clean job labels
train_labels = []

# print "Cleaning and parsing the training set job labels...\n"

# Loop over each job; create an index i that goes from 0 to the length
# of the job list 
for i in range( 0, num_data ):
    # Call our function for each one, and add the result to the list of

    # If the index is evenly divisible by 1000, print a message
    if( (i+1)%1000 == 0 ):
        print ("Job %d of %d\n" % ( i+1, num_data ))                                                             

    # clean out NaN
    if type(train["tags"][i]) != str:
        train["tags"][i] = ''
    train_labels.append(train["tags"][i]) 

# print "Sample training labels: \n", train_labels[0]

Job 1000 of 4375

Job 2000 of 4375

Job 3000 of 4375

Job 4000 of 4375



In [12]:
print ("Creating the bag of words for job labels...\n")
from sklearn.feature_extraction.text import CountVectorizer

# Keep words hyphenated
pattern = "(?u)\\b[\\w-]+\\b"

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer2 = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 5000,
                             token_pattern=pattern) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_labels = vectorizer2.fit_transform(train_labels)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_labels = train_data_labels.toarray()

print ("Sample vectorized training labels: \n",train_data_labels[0])

Creating the bag of words for job labels...

Sample vectorized training labels: 
 [0 0 1 0 0 0 0 1 0 0 0 1]


In [13]:
# Evaluate a few classifiers, then choose one for optimizing next

# Import the desired classifiers, splitters, metrics etc.

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report

from time import time

# All classifiers are named clf for compatibility with tester.py
# Comment out ('#') all classifiers other than the desired one

#clf = DecisionTreeClassifier(random_state=42)
#clf = KNeighborsClassifier()
clf = RandomForestClassifier(random_state=42)

# Split data into training and testing sets, using 30% split

t0 = time()

features_train, features_test, labels_train, labels_test = \
    train_test_split(train_data_features, train_data_labels, test_size=0.3, random_state=42)
    
clf.fit(features_train,labels_train)
labels_train_est = clf.predict(features_train)
labels_pred = clf.predict(features_test)

In [14]:
print("Results for Training: \n", classification_report(labels_train, labels_train_est))
print("Results for Testing: \n", classification_report(labels_test, labels_pred))
print("total train/test/prediction time:", round(time()-t0, 3), "s")

Results for Training: 
              precision    recall  f1-score   support

          0       1.00      0.82      0.90       233
          1       1.00      0.94      0.97       719
          2       1.00      0.87      0.93       430
          3       1.00      0.83      0.91       153
          4       1.00      0.97      0.99       680
          5       1.00      0.92      0.96       610
          6       1.00      0.93      0.96       317
          7       1.00      0.87      0.93       382
          8       1.00      0.77      0.87        48
          9       1.00      0.88      0.94       225
         10       1.00      0.88      0.94       451
         11       1.00      0.89      0.94       529

avg / total       1.00      0.91      0.95      4777

Results for Testing: 
              precision    recall  f1-score   support

          0       0.00      0.00      0.00        98
          1       0.53      0.15      0.24       324
          2       0.48      0.06      0.10      

C:\Users\PhucBuiLocal\PycharmProjects\ML\venv\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [15]:
from sklearn.externals import joblib
joblib.dump(clf,"train.dat")

['train.dat']

In [71]:
from nltk.tokenize import sent_tokenize, word_tokenize
import re
import nltk
nltk.download('punkt')
data = """Mô tả
- Vận chuyển sản phẩm của công ty đến khách hàng theo sự phân công của quản lý.
- Lưu giữ và vận chuyển hàng hóa cẩn thận.
- Giao đầy đủ hàng hóa, hóa đơn chứng từ cho khách hàng.
- Giải thích thắc mắc của khách hàng liên quan tới sản phẩm.
Địa chỉ làm việc: số 7, đại lộ độc lập, KCN Sóng Thần, Dĩ An, Bình Dương
Yêu cầu
- Nam , có xe gắn máy.
- Nhanh nhẹn, sức khỏe tốt
- Chăm chỉ, cẩn thận, nghiêm túc với công việc
- Ưu tiên ứng viên đã có kinh nghiệm, thành thạo đường tại khu vực
- Giao tiếp lịch sự, nhã nhặn
Quyền lợi
- Môi trường làm việc năng động, nhiều cơ hội và thách thức;
- Được tham gia BHXH, BHYT, BHTN;
- Được Cty mua bảo hiểm rủi ro 24 / 24;
- Các chế độ theo quy định của Luật Lao động và Công ty (sinh nhật, hiếu, hỉ, ốm đau,...)
Hồ sơ Ưu tiên nộp hồ sơ trực tuyến qua hệ thống của Timviecnhanh.com
Hoặc gửi CV mô tả quá trình học tập và làm việc về email liên hệ."""

data1 = """Mô tả 	

- Giao hàng bằng xe máy trong khu vực Q1, Q2, Q3, Q4, Q5, Q7, Bình Thạnh, Phú Nhuận.
- Chủ yếu là giao đồ ăn nhẹ nhàng, khách lịch sự hay “tip”.
- Ca 15 ngày: 10h30 - 23h30, ngày làm ngày nghỉ xen kẽ.
- Ca tối: 18h30 - 23h30.
Yêu cầu 	

- Trình độ bằng cấp: Không yêu cầu
- Rành đường thành phố hoặc có tinh thần học hỏi, chịu khó sẽ training từ từ.
- Chăm chỉ, thật thà.
- Nhiệt tình, có tinh thần học hỏi, chăm chỉ, đúng giờ.
- Độ tuổi tuyển dụng: Dưới 23 tuổi
- Giới tính: Nam (Không tuyển nữ)
- Ưu tiên có nhà HCM hoặc không bị giới hạn giờ giấc nhà trọ.
- Ưu tiên khu vực gần cửa hàng ở phường 22 Bình Thạnh.
Quyền lợi 	

- Thu nhập tối thiểu từ 4.500.000 trở lên, tiền tip cao, công việc đơn giản.
- Hỗ trợ chi phí điện thoại liên lạc, tiền thối và đào tạo đầy đủ các kĩ năng làm việc nhóm, giao tiếp, xử lý vấn đề.
- Có các chính sách thưởng hàng tháng.
- Du lịch thường niên.
- Cơ hội thăng tiến công bằng.
- Môi trường trẻ trung năng động, vui vẻ đoàn kết, có làm có chơi có anh em bạn bè đồng nghiệp tận tình hỗ trợ.
- Các sếp cực kì lạ và độc.
- Đồng nghiệp cực kì đáng yêu và chuyên nghiệp.
Hồ sơ 	

- CMND photo
- Sơ yếu lí lịch
- Đơn xin việc
- Bản sao hộ khẩu
(Tất cả giấy tờ không cần công chứng)"""

data2 = """
Mô tả 	

- Đến ngân hàng giao nhận chứng từ.
- Lên hãng tàu lấy B/L, D/ O, hóa đơn, đi giao nhận chứng từ liên quan.
- Theo dõi, giám sát quy trình giao nhận hàng hàng hóa.
- Xử lý các công việc liên quan.
Yêu cầu 	
- Nam
- Có kinh nghiệm trong lĩnh vực giao nhận Xuất nhập khẩu.
- Sẽ được training thêm khi vào làm.
- Có phương tiện đi lại.
- Tiếp thu nhanh, nhiệt tình trong công việc, hòa đồng, trung thực.
Quyền lợi 	

- Được hưởng đầy đủ quyền lợi của người lao động theo luật hiện hành.
- Được hưởng chế độ thưởng lễ, Tết theo kết quả kinh doanh của công ty.
- Được tham gia đào tạo nâng cao chuyên sâu chuyên môn và kỹ năng mềm.
- Cơ hội phát triển bản thân và thăng tiến trong tổ chức.
- Môi trường làm việc năng động, thân thiện.
- Lương: 7 triệu + phụ cấp.
Hồ sơ 	

- Ưu tiên nộp hồ sơ qua hệ thống timviecnhanh.com
- Hoặc gửi trực tiếp về email liên hệ.
"""
raw_data = [data, data1, data2]
raw_label = [1, 0, 0]
raw_train = []
for  d in raw_data:
    words = job_to_words(data, "vietnamese")
    raw_train.append(words)
train_data_labels = raw_label

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PhucBuiLocal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [72]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer1 = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = 2000,
                              ngram_range = (1,4)) 

# fit_transform() does two functions: First, it fits the model
# and learns the vocabulary; second, it transforms our training data
# into feature vectors. The input to fit_transform should be a list of 
# strings.
train_data_features = vectorizer1.fit_transform(raw_train)

# Numpy arrays are easy to work with, so convert the result to an 
# array
train_data_features = train_data_features.toarray()